Tenemos 4 carpetas de matrices (cada una con un documento pkl por anyo(de 2000 a 2019))
   - 2 de informacion Topográfica (ya creadas): 
       -matricesFuegos 
       -matricesNoFuegos
       
   - 2 de informacion meteorologica (pendiente de santi)
        -matricesFMeteo
        -matricesNoFMeteo

Con las 2 carpetas de informacion meteorologica vendra el csv dfMETEOtotal (por anyos)

1. crear una funcion que compruebe que el número de entradas en el csv por anyo corresponde al numero de matrices ya guradadas por anyo. y si coincide con el numero de matrices de no fuego ese anyo. Si no coincide hay repasar el df para ver cuales hay que eliminar.


2. se crean 2 nuevas carpetas: 
        -m_F27x27x8
        -m_Nf27x27x8
        

Para cada carpeta, 
el registro de matrices está controlado desde: 
dfMETEOtotal (por anyos)... 
con dfFuegosConInfoTopo.csv (con 6277 puntos de ignicion con informacion topografica) nos podemos asegurar de que los puntos corresponden a sus matrices. 


# Crear una funcion que recoja las matrices de una y otra carpeta y las junte.

# A partir de cada matriz combinada se extraen 9 matrices. 

y nos aseguramos de que cada matriz con fuego tiene su coontraparte en matriz sin fuego

Finalmente por cada matriz de 27x27x8 sacamos 9 matrices de 25x25x8


cada carpeta (fuego y no fuego) deberia tener ej 6277 x 9 = 54000 matrices para entrenar la red.

In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import shapely.geometry
from shapely.geometry import Polygon
from shapely.ops import cascaded_union
from netCDF4 import Dataset
from PIL import Image

import datetime
import pickle


# 1. Fusionamos topografia y meteorologia de los puntos con fuego

In [2]:
# VERSION FUNCION!


# 1. Fusionamos topografia y meteorologia de los puntos con fuego

# primero intentamos generar matrices con datos TOPOGRAFICOS de 27x27 (13.5km de lado) de todos los puntos de ignicion 
# en algunos casos movimos el centro (el punto de ignicion hasta 500m) para asegurar los 13.5 km de lado con datos
# el resultado fue este df. 

def recogerMatTopoMeteo(seleccionAnyo=2019):

    direccion=('DATOSDEENTRADAPARAmezclar/dfFuegosConInfoTopo.csv')
    matFuegTopo2000A2019 = pd.read_csv(direccion, index_col=0)

    date_time_obj = datetime.datetime.strptime(str(seleccionAnyo), '%Y')

    matFuegTopo2000A2019["fecha"]= pd.to_datetime(matFuegTopo2000A2019["fecha"])

    matFuegTopo2000A2019=matFuegTopo2000A2019[matFuegTopo2000A2019['fecha']>date_time_obj ]

    print(f"En el anyo {seleccionAnyo} se contablizaron {len(matFuegTopo2000A2019)} fuegos")


    #estas 247 matrices con informacion topografica estan guardadas ordenadamente en el siguiente archivo
    direccion='DATOSDEENTRADAPARAmezclar/matricesTopo/matSI_f_TOPO_00a20/matricesFuegos'+str(seleccionAnyo)+'.pkl'
    with open(direccion, 'rb') as f:
        mat_Fuego_topo_2019 = pickle.load(f)   
    print(f'El mismo anyo tiene {len(mat_Fuego_topo_2019)} matrices TOPOGRAFICAS de Fuegos guradadas')


    # paralelamente buscamos puntos random sin fuegos (ni quemados en anyos anteriores)
    # y creamos un listado holgado de puntos random 
    # de esto seleccionamos los que tienen informacion topografica de 13.5km de lado para rellenar la matriz 27x27
    # las matrices se guardaron y anotaron en el siguiient df.
    direccion='DATOSDEENTRADAPARAmezclar/matricesTopo/dfFinalNOfuego/dfFinalNOfuego'+str(seleccionAnyo)+'.csv' 
    matNoFuegTopo2019 = pd.read_csv(direccion, index_col=0)

    #de los 494 puntos random, 367 tenian suficientes datos para generera matrices topograficas
    #matNoFuegTopo2000A2019=matNoFuegTopo2000A2019[matNoFuegTopo2000A2019['datos']!='Sin datos']
    antes=len(matNoFuegTopo2019)
    matNoFuegTopo2019=matNoFuegTopo2019[matNoFuegTopo2019['datos']!='Sin datos']
    print(f'De los {antes} puntos random, solo {len(matNoFuegTopo2019)} tenian suficientes datos para generera matrices topograficas')

    #cada una de las coordenadas anteriores coorresponde a una matriz dentro del archivo matricesNOFuegos2019.pkl

    #estas 494 matrices con informacion topografica estan guardadas ordenadamente en el siguiente archivo
    direccion='DATOSDEENTRADAPARAmezclar/matricesTopo/matNO_f_TOPO_00a20/matricesNOFuegos'+str(seleccionAnyo)+'.pkl'
    with open(direccion, 'rb') as f:
        mat_NOFuego_topo_2019 = pickle.load(f)   
    print(f'El mismo anyo tiene {len(mat_NOFuego_topo_2019)} matrices TOPOGRAFICAS de NO Fuegos guradadas')


    # A partir de la informacion anterior, se recoge en remoto (a traves de tecnosylva) la informacion metereologica.

    #En caso de que el punto de ignicion no tenia informacion meterologica este se descarta 
    #para posteriormente ignorar la mattriz de informacion topografica guardada (en funcion de su posicion)

    #En caso de que el punto de ignicion tenga datos, se busca entre las coordenadas sin fuego alguna 
    #que con los datos meteorologicos del mismo dia pueda rellenar una matriz de 27x27 de 13.5km de lado 


    direccion=('DATOSDEENTRADAPARAmezclar/dfMETEOtotal/dfMETEOtotal'+str(seleccionAnyo)+'.csv')
    dfmatricesTopo_Meteo2019 = pd.read_csv(direccion, index_col=0)

    #recogemos el df con los datos recogidos por Tecnosilva 
    print(f'El DF de datos recogiso por tecnosilva contiene {len(dfmatricesTopo_Meteo2019)} lineas y {len(dfmatricesTopo_Meteo2019.columns)} columnas'  )

    #print(f" total mat meteo : {len(dfmatricesTopo_Meteo2019[dfmatricesTopo_Meteo2019['meteo_F']=='Con datos'])} con fuegos")
    #print(f" total mat meteo : {len(dfmatricesTopo_Meteo2019[dfmatricesTopo_Meteo2019['met_NoF']=='Con datos'])} sin fuegos")




    direccion='DATOSDEENTRADAPARAmezclar/matricesMeteo/matricesFMeteo/matricesFMeteo'+str(seleccionAnyo)+'.pkl' 
    with open(direccion, 'rb') as f:
        mat_Fuego_METEO_2019 = pickle.load(f)   
    print(f'El mismo anyo tiene {len(mat_Fuego_METEO_2019)} matrices METEOROLOGICAS de Fuegos guradadas')


    direccion='DATOSDEENTRADAPARAmezclar/matricesMeteo/matricesNoFMeteo/matricesNoFMeteo'+str(seleccionAnyo)+'.pkl' 
    with open(direccion, 'rb') as f:
        mat_NOFuego_METEO_2019 = pickle.load(f)   
    print(f'El mismo anyo tiene {len(mat_NOFuego_METEO_2019)} matrices METEOROLOGICAS de NO Fuegos guradadas')

    
    
    
    
    return dfmatricesTopo_Meteo2019, mat_Fuego_topo_2019, mat_Fuego_METEO_2019, mat_NOFuego_topo_2019,mat_NOFuego_METEO_2019

'''
por cada dato de dfmatricesTopo_Meteo2019 con met_NoF == 'Con datos' en funcion de la posicion recogemos: ej. posicion 0 matriz 0 del pickle de datos topograficos de fuegos y la juntamos con la primera de las 228 matrices del pickle de datos meteorologicos de fuegos: mat_Fuego_METEO_2019

A continuacion:

PARA LOCALIZAR LAS MATRICES DE NO FUEGOS TOPOGRAFICAS QUE JUNTAREMOS CON LAS METEREOLOGICAS
# buscamos los valores LAS COLUMNAS LO_NoF   LA_NoF de dfmatricesTopo_Meteo2019 
# que coinciden con los valores de las columnas long_F     lat_F del df matNoFuegTopo2000A2019
# al encontrar la coincidencia ya podemos usar 
#la posicion de matNoFuegTopo2000A2019 para seleccionar la matriz correspondiente en 

y la juntamos con la primera de las 228 matriz del pickle de datos meteorologicos de NO fuegos: mat_NOFuego_METEO_2019

'''


# ESTA FUNCION COMBINA LAS MATRICES METEREOLOGICAS (DE 27X27X4) Y TOPOGRAFICAS (DE 27X27X4)
# GENERANDO UNA TUPLA DE FUNCIONES DE 27X27X8.
#LA FUNCION NECESITA:
    #dfmatricesTopo_Meteo2019 
    #mat_Fuego_topo_2019
    #mat_Fuego_METEO_2019
    #matNoFuegTopo2000A2019
    #mat_NOFuego_METEO_2019

def matrizAnual27(dfmatricesTopo_Meteo2019,mat_Fuego_topo_2019,mat_Fuego_METEO_2019,mat_NOFuego_topo_2019,mat_NOFuego_METEO_2019):
    fu27x27x8=[]
    noFu27x27x8=[]

    meteoConDatos=0
    for x in range(len(dfmatricesTopo_Meteo2019)):
        # seleccionamos las posiciones de dfmatricesTopo_Meteo2019 que tienen datos

        if dfmatricesTopo_Meteo2019['met_NoF'][x]=='Con datos':
            if dfmatricesTopo_Meteo2019['LO_NoF'][x] != 0:
                contar=-1
                match=False
                while match==False and contar < len(dfmatricesTopo_Meteo2019)-1:
                    contar+=1
                    #print(x)
                    #print(contar)        
                    if (dfmatricesTopo_Meteo2019['LO_NoF'][x] == dfmatricesTopo_Meteo2019['long_F'][contar]):
                        if dfmatricesTopo_Meteo2019['LA_NoF'][x] == dfmatricesTopo_Meteo2019['lat_F'][contar]:
                            match=True
                            print('\nNEXT')      
                        else:
                            print(f'Se ha movido la latitud en {x}_______________________________')                 
                    else:
                        match=False
                    #print(match)        


            fu27x27x8.append(mat_Fuego_topo_2019[x]+mat_Fuego_METEO_2019[meteoConDatos])
            noFu27x27x8.append(mat_NOFuego_topo_2019[contar]+mat_NOFuego_METEO_2019[meteoConDatos])
            #print(f'Se ha guardado la posicion {x} del df mat_Fuego_topo_2019:')
            #print(dfmatricesTopo_Meteo2019[x:x+1])
            #print(f'Que corresponde a la posicion {meteoConDatos} de las matrices meteorologicas\nY a la posicion {contar} de las matNoFuegTopo2000A2019:')
            #print(dfmatricesTopo_Meteo2019[contar:contar+1])
            meteoConDatos+=1
            
    return fu27x27x8,noFu27x27x8



#DE CADA UNA DE LAS (228X2) MATRICES DE 27X27X8 QUEREMOS SACAR 9 MATRICES DE 25X25


#CASO 1 MATRIZ DE 27X27
def dataAugmentationX9(m27):
    matiZero=m27
    
    #todos los cortes para que resulten 9 matrices de 25x25
    equis=[[0,25],[1,26],[2,27]]
    yes=[[0,25],[1,26],[2,27]]

    CAPAS=[] 
    for capa27x27 in matiZero:
        M25x25xCAPA=[]
        for x in range(3):
            x0=equis[x][0]
            x1=equis[x][1]
            for y in range(3):            
                y0=yes[y][0]
                y1=yes[y][1]
                #print(matriz.shape)
                recortada=capa27x27[x0:x1,y0:y1]
                #print(recortada.shape)
                M25x25xCAPA.append(recortada)  #tendremos 9 
                    #esto da como resultado un 
        #print(f'De esta capa/caracteristica de 27x27 se han recogido {len(M25x25xCAPA)} matrices de 25x25') #deberiamos tener las 9 matrices de 25x25 
        CAPAS.append(M25x25xCAPA)


    matriz8capasFinal=[] #listado de matrices 9 finales 
    for caracteristica in range(9):
        juntamosCapasDeUNAmatriz=[]
        for posicion in range(8): 
            juntamosCapasDeUNAmatriz.append(CAPAS[posicion][caracteristica])
        matriz8capasFinal.append(juntamosCapasDeUNAmatriz)

    #print(f'El resultado son:\n{len(matriz8capasFinal)} (9)matrices de {len(matriz8capasFinal[0])} (8)dimensiones cada matriz de {matriz8capasFinal[0][0].shape} de 25x25 cada dimension')
    return matriz8capasFinal


def pipelineA25x25x8(anyo):

    dfmatricesTopo_Meteo2019, mat_Fuego_topo_2019, mat_Fuego_METEO_2019, mat_NOFuego_topo_2019,mat_NOFuego_METEO_2019=recogerMatTopoMeteo(anyo)

    fu27x27x8,noFu27x27x8=matrizAnual27(dfmatricesTopo_Meteo2019,mat_Fuego_topo_2019,mat_Fuego_METEO_2019,mat_NOFuego_topo_2019,mat_NOFuego_METEO_2019)

    print(f'\nDel anyo {anyo} se han obtenido {len(fu27x27x8)} la matrices con fuego y {len(noFu27x27x8)} matrices SIN fuego')

    mCON_F_25_2019=[]
    for m27 in fu27x27x8:
        mCON_F_25_2019.extend(dataAugmentationX9(m27))
        
    direccion=('DATOSLISTOS25X25/m_F4cnn/mCON_F_25_'+str(anyo)+'.pkl')
    with open(direccion, 'wb') as f:
        pickle.dump(mCON_F_25_2019, f)
        
        
        
    mSIN_F_25_2019=[]
    for m27 in noFu27x27x8:
        mSIN_F_25_2019.extend(dataAugmentationX9(m27))
        
    direccion=('DATOSLISTOS25X25/m_NoF4cnn/mSIN_F_25_'+str(anyo)+'.pkl')
    with open(direccion, 'wb') as f:
        pickle.dump(mSIN_F_25_2019, f)    
        

    print(f'Finalmente, de {anyo} se gurardan {len(mCON_F_25_2019)} matrices de 25x25x8 CON FUEGO Y  {len(mSIN_F_25_2019)} SIN FUEGO')
    return len(mCON_F_25_2019)

In [3]:
anyos = list(range(2000,2020))
totalEntradas=0 
for anyo in anyos:
    totalEntradas+=pipelineA25x25x8(anyo)
    
print(f'\nEn total hay {(totalEntradas/9)*2} matrices de fuegos y no fuegos.\nUna vez aumentadas {(totalEntradas)*2}')

En el anyo 2000 se contablizaron 5186 fuegos
El mismo anyo tiene 151 matrices TOPOGRAFICAS de Fuegos guradadas
De los 302 puntos random, solo 243 tenian suficientes datos para generera matrices topograficas
El mismo anyo tiene 243 matrices TOPOGRAFICAS de NO Fuegos guradadas
El DF de datos recogiso por tecnosilva contiene 151 lineas y 10 columnas
El mismo anyo tiene 28 matrices METEOROLOGICAS de Fuegos guradadas
El mismo anyo tiene 28 matrices METEOROLOGICAS de NO Fuegos guradadas

Del anyo 2000 se han obtenido 28 la matrices con fuego y 28 matrices SIN fuego
Finalmente, de 2000 se gurardan 252 matrices de 25x25x8 CON FUEGO Y  252 SIN FUEGO
En el anyo 2001 se contablizaron 5036 fuegos
El mismo anyo tiene 179 matrices TOPOGRAFICAS de Fuegos guradadas
De los 358 puntos random, solo 281 tenian suficientes datos para generera matrices topograficas
El mismo anyo tiene 281 matrices TOPOGRAFICAS de NO Fuegos guradadas
El DF de datos recogiso por tecnosilva contiene 179 lineas y 10 columnas
El


Del anyo 2012 se han obtenido 49 la matrices con fuego y 49 matrices SIN fuego
Finalmente, de 2012 se gurardan 441 matrices de 25x25x8 CON FUEGO Y  441 SIN FUEGO
En el anyo 2013 se contablizaron 2067 fuegos
El mismo anyo tiene 242 matrices TOPOGRAFICAS de Fuegos guradadas
De los 484 puntos random, solo 375 tenian suficientes datos para generera matrices topograficas
El mismo anyo tiene 375 matrices TOPOGRAFICAS de NO Fuegos guradadas
El DF de datos recogiso por tecnosilva contiene 242 lineas y 10 columnas
El mismo anyo tiene 43 matrices METEOROLOGICAS de Fuegos guradadas
El mismo anyo tiene 43 matrices METEOROLOGICAS de NO Fuegos guradadas

Del anyo 2013 se han obtenido 43 la matrices con fuego y 43 matrices SIN fuego
Finalmente, de 2013 se gurardan 387 matrices de 25x25x8 CON FUEGO Y  387 SIN FUEGO
En el anyo 2014 se contablizaron 1824 fuegos
El mismo anyo tiene 183 matrices TOPOGRAFICAS de Fuegos guradadas
De los 366 puntos random, solo 278 tenian suficientes datos para generera mat

# COMPROBACION!!!!!!!!

In [4]:
#para abrirlos
direccion=('DATOSLISTOS25X25/m_NoF4cnn/mSIN_F_25_2019.pkl')
with open(direccion, 'rb') as f:
    SinFuego2019 = pickle.load(f)  
    
print(len(SinFuego2019))

SinFuego2019[0][0][0]

2205


array([0.66827703, 0.62218892, 0.59199327, 0.55226215, 0.53478045,
       0.54272668, 0.57451157, 0.61742119, 0.62218892, 0.63808137,
       0.64761684, 0.63490288, 0.6285459 , 0.6142427 , 0.57451157,
       0.57927931, 0.58563629, 0.57451157, 0.56974384, 0.54431592,
       0.50935253, 0.48392461, 0.47438914, 0.4727999 , 0.47121065])

In [5]:
#para abrirlos
direccion=('DATOSLISTOS25X25/m_F4cnn/mCON_F_25_2019.pkl')
with open(direccion, 'rb') as f:
    ConFuego2019 = pickle.load(f)  
    
print(len(ConFuego2019))

ConFuego2019[0][0][0]

2205


array([0.13231539, 0.12881243, 0.1282286 , 0.12764478, 0.12297417,
       0.1258933 , 0.13056391, 0.1346507 , 0.13990513, 0.14457574,
       0.1568361 , 0.17435089, 0.18836273, 0.17435089, 0.16384202,
       0.16909645, 0.16033906, 0.18252446, 0.1790215 , 0.16617732,
       0.136986  , 0.13406687, 0.16384202, 0.18661125, 0.18252446])